## 1.数据预处理

Data Preprocessing

### 1.1 配置参数 
> 需根据实际文件路径修改
- 需要解析 PDF 文件路径
- 输出文件 CSV 路径
- 输出文件 CSV 表头

In [34]:
def prepare_params():
    pdf_path = "./assets/2024年四川省重点项目名单.pdf"  
    output_csv = "./outputs/2024年四川重点项目解析结果.csv"
    target_headers = ["总序号", "分序号", "项目名称", "建设地址"]

    return pdf_path, output_csv, target_headers

### 1.2 使用 pdfplumber 解析 PDF 页面数据

In [35]:
import pdfplumber

def extract_tables_from_pdf(pdf_path, target_headers):
    all_tables_data = []  # 存储所有页面的表格数据
    header_count = 0  # 计数表头出现次数，用于去重

    with pdfplumber.open(pdf_path) as pdf:
        # 获取实际总页数
        total_pages = len(pdf.pages)  
        
        # 遍历PDF的每一页
        for page_num in range(0, total_pages):
            page = pdf.pages[page_num]
            print(f"正在解析第{page_num + 1}页...")
            
            # 提取当前页所有表格
            tables = page.extract_tables(
                table_settings={
                    "vertical_strategy": "lines",  # 按竖线识别列边界
                    "horizontal_strategy": "lines",  # 按横线识别行边界
                    "text_y_tolerance": 10,  # 文本垂直方向容错
                    "text_x_tolerance": 5,   # 文本水平方向容错
                }
            )
            
            # 遍历当前页的每个表格
            for table in tables:
                if not table:  # 跳过空表格
                    continue
                
                # 使用新函数处理表格数据
                header_count, table_data = process_table_rows(table, target_headers, header_count)
                # 将处理后的数据添加到总数据列表中
                all_tables_data.extend(table_data)
    
    return all_tables_data

def process_table_rows(table, target_headers, header_count):
    """
    处理单个表格的所有行数据
    
    Args:
        table: 单个表格的数据（二维列表）
        target_headers: 目标表头列表
        header_count: 已经遇到的表头数量
        
    Returns:
        tuple: (处理后的表头计数, 该表格处理后的数据列表)
    """
    table_data = []  # 存储当前表格处理后的数据
    current_header_count = header_count  # 本地副本用于处理

    for row in table:
        # 清理每个单元格的多余空格和换行符
        cleaned_row = [cell.strip().replace("\n", "") if cell else "" for cell in row]
        
        # 去重重复表头
        is_header = all(col in cleaned_row for col in target_headers)
        if is_header:
            current_header_count += 1
            if current_header_count > 1:  # 跳过第2次及以后的重复表头
                continue
            else:
                table_data.append(cleaned_row)  # 保留第一次出现的表头
        else:
            # 筛选有效数据
            total_seq = cleaned_row[0] if len(cleaned_row) > 0 else ""
            project_name = cleaned_row[2] if len(cleaned_row) > 2 else ""
            
            # 排除分类标题
            if (total_seq.isdigit() or total_seq == "") and project_name != "" and "项目" in project_name:
                table_data.append(cleaned_row)
                
    return current_header_count, table_data

### 1.3 使用 pandas 数据清洗

转换为 DataFrame 并清洗

In [36]:
import pandas as pd

def convert_to_dataframe(all_tables_data):
    # 将列表转为 DataFrame，用目标表头命名列
    df = pd.DataFrame(all_tables_data[1:], columns=all_tables_data[0])  # 0行是表头，1行后是数据

    # 进一步清洗：删除全空行、处理缺失值
    df = df.dropna(how="all")  # 删除全空行

    # 确保列存在再进行填充操作
    if "总序号" in df.columns:
        df["总序号"] = df["总序号"].fillna("")  # 总序号为空的填充为空字符串（如子项目）
    if "分序号" in df.columns:
        df["分序号"] = df["分序号"].fillna("")  # 分序号为空的填充为空字符串

    # 添加“项目类型”列
    if "总序号" in df.columns:
        df["项目类型"] = df["总序号"].apply(lambda x: "续建" if (str(x).isdigit() and int(x) <= 504) else "新开工")
    else:
        print("警告：未找到'总序号'列，跳过'项目类型'生成。")
        df["项目类型"] = ""
    
    return df

### 1.4 保存结果（CSV）

In [37]:
def save_results(df, output_csv):
    # 保存为 CSV（兼容更多工具，如 Excel、Python读取）
    # utf-8-sig 解决中文乱码
    df.to_csv(output_csv, index=False, encoding="utf-8-sig") 
    return output_csv


### 数据处理主函数运行 (main)
> 组织各个步骤函数运行

In [40]:
from datetime import datetime

def mainDataPreprocess():
    # 配置参数
    pdf_path, output_csv, target_headers = prepare_params()
    
    # 提取表格数据
    all_tables_data = extract_tables_from_pdf(pdf_path, target_headers)
    
    # 转换为DataFrame
    df = convert_to_dataframe(all_tables_data)
    
    # 保存结果
    saved_path = save_results(df, output_csv)

    # 获取当前时间
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 输出结果信息
    print(f"解析完成({current_time})！共提取{len(df)}条项目数据，已保存至：")
    print(f"- CSV: {saved_path}")
    
    # 打印前5条数据验证结果
    print("\n解析结果预览（前5条）：")
    print(df.head())

# 运行主程序
if __name__ == "__main__":
    mainDataPreprocess()

正在解析第1页...
正在解析第2页...
正在解析第3页...
正在解析第4页...
正在解析第5页...
正在解析第6页...
正在解析第7页...
正在解析第8页...
正在解析第9页...
正在解析第10页...
正在解析第11页...
正在解析第12页...
正在解析第13页...
正在解析第14页...
正在解析第15页...
正在解析第16页...
正在解析第17页...
正在解析第18页...
正在解析第19页...
正在解析第20页...
正在解析第21页...
正在解析第22页...
正在解析第23页...
正在解析第24页...
正在解析第25页...
正在解析第26页...
正在解析第27页...
正在解析第28页...
正在解析第29页...
正在解析第30页...
正在解析第31页...
正在解析第32页...
正在解析第33页...
正在解析第34页...
正在解析第35页...
正在解析第36页...
正在解析第37页...
正在解析第38页...
正在解析第39页...
正在解析第40页...
正在解析第41页...
正在解析第42页...
正在解析第43页...
正在解析第44页...
正在解析第45页...
正在解析第46页...
正在解析第47页...
正在解析第48页...
正在解析第49页...
正在解析第50页...
正在解析第51页...
正在解析第52页...
正在解析第53页...
正在解析第54页...
正在解析第55页...
正在解析第56页...
正在解析第57页...
正在解析第58页...
正在解析第59页...
正在解析第60页...
正在解析第61页...
正在解析第62页...
正在解析第63页...
正在解析第64页...
正在解析第65页...
正在解析第66页...
正在解析第67页...
正在解析第68页...
正在解析第69页...
正在解析第70页...
正在解析第71页...
正在解析第72页...
正在解析第73页...
正在解析第74页...
正在解析第75页...
正在解析第76页...
警告：未找到'总序号'列，跳过'项目类型'生成。
解析完成(2025-11-02 23:36:33)！共提取481条项目数据，已保存至：
- CSV: ./outputs/2024年四川重点项目